## Least Cost Pipeline Construction
This notebook imports the cost distance stack generated by `Create-Cost-Stack.ipynb` and a pipeline raster to compute a least cost "feeder" pipeline configuration. The workflow for this process is:
1. Import the stack of cost distance layers (one for each biogas source): $arr\_stack$.
1. Import the pipeline raster, setting non-pipeline cells to NaN: $arr\_pipeline$.
1. Determine which pixel among the pipeline pixels has the least cost among all farm cost distances rasters. This will serve as the location of the connection point to the existing pipeline: $C_0$.
1. Determine which farm is the source of this minimum point, done by finding which layer (in the stack of cost distance rasters) has the minimum value at that location. This represents the least cost biogas source: $F_0$.
1. Compute the least cost path connecting that farm ($F_0$) to the connection point ($C_0$): $LCP_0$.
1. Update the connection point layer ($C_0$) to include the least cost path ($LCP_0$): $Pipes_0$.
1. Remove the layer associated with $F_0$ from the stack of cost distance rasters ($arr\_CDsk$) and repeat steps 4-7:
    * Locate the minimim value among all remaining cost distance rasters to cells in the Pipes layer ($C_i$)...
    * Idenfity the source farm associated with this minimum ($F_i$)...
    * Compute the least cost path from from $F_i$ to $C_i$...
    * Update the connection point layer ($Pipes_{i+1}$)...

In [ ]:
#Import packages
import numpy as np
import pandas as pd
from skimage import graph
from osgeo import gdal
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Read the cost and cost distance stacks
arrCosts = np.load('../data/DuplinCostStack.npy')
arrStack = np.load('../data/DuplinStack.npy')

In [ ]:
#Read in the pipeline rasters
ds =  gdal.Open('../data/processed/duplin_pipe_diameter.tif')
#Get the raster projection
ds_prj = ds.GetProjection()
#Get raster attributes (used in MCP analysis)
llx, x_size, x_angle, lly, y_angle, y_size = ds.GetGeoTransform()
#Extract Band1 as the cost array; divide by 100 to scale as a percentage
arrPipesAll = np.array(ds.GetRasterBand(1).ReadAsArray())
#Convert all pipe pixels to 1, others to -1
arrPipesAll[arrPipesAll >= 0] = 1
arrPipesAll[arrPipesAll < 0] = -1
#Convert to a masked array (ignoring all values < 0, which are NoData)
arrPipes = np.ma.masked_array(arrPipesAll, mask=arrPipesAll < 0)

## Functions

In [ ]:
def getMin(array_stack,array_mask,verbose=False):
    '''Returns the layer and row/col index of the pixel in a stack of layers containing the 
       overall minimum value found among cells within the supplied mask array.
    
    Arguments:
     array_stack (3d array): Stack of cost distance layers from each biogas source
     array_mask (2d array): Layer of pixels from which min values should be returned
     verbose (Boolean): Set to true to display processing messages
    
    Returns (tuple):
     [1] (integer) Index of the layer in the array_stack containing the minimum values
     [2] (tuple) Row/column coordinates of the pixel containing the minimum value.
    
    '''
    #Reduce the stack of cost distance layers to a layer of the minimum value at each location
    arrMin = np.amin(array_stack,axis=0)
    #Mask the values so only pipeline pixels are kept
    arrMin_masked = np.ma.masked_array(arrMin, mask=array_mask != 1)
    #Find the min value of the pipeline pixels
    minValue = np.amin(arrMin_masked)
    #Determine the row and column where the min occurs
    rMin,cMin = np.where(arrMin_masked == minValue)
    #Extract values from value arrays
    rowMin = rMin[0];  colMin = cMin[0]
    #Find which layer has the min value at this location in the stack of layers
    layers = np.where(array_stack[:,rowMin,colMin] == minValue)[0]
    #if len(layers) > 1: layerIdx = layers[0]
    #else: layerIdx = [layers]
    layerIdx = layers[0]
    if verbose: print("Link to LCP row/col ({},{} (Layer:{}))".format(rowMin,colMin,layerIdx))
    return(layerIdx,(rowMin,colMin))

In [ ]:
def getSourceCoordinates(array_stack,layer_id,verbose=False):
    '''Returns the row and column index for a biogas source in the provided layer
    
    Args: 
     array_stack (3d array): Stack of cost distance layers from each biogas source
     layer_id (integer): Index value of the layer containing the biogas source
     verbose (Boolean): Set to true to display processing messages
    
    Returns (tuple): 
     The Row/column coordinates of the pixel containing the minimum 
     value in the layer, i.e., the biogas source.
    
    '''
    #Get the layer from the stack corresponding to the source
    arrSource = array_stack[layer_id]
    #Find the minimum of that layer
    minValue = np.amin(arrSource)
    #Get the row and columns corresponding to that value
    rFarm,cFarm = np.where(array_stack[layer_id,:,:]==minValue)
    #Convert value arrays to just values
    theRow = rFarm[0]; theCol = cFarm[0]
    if verbose: print("Source occurs at ({},{})".format(theRow,theCol))
    #Return coords as a tuple
    return(theRow,theCol)

In [ ]:
def getLCP(array_costs,layer_id,source_coords,pipe_coords,x_size=500,y_size=500):
    '''Returns a least cost layer between biogas source and pipeline coordinates
       for a given layer using the cost surface for a specific biogase source among
       the stack of biogas cost surface layers. 
       
    Args:
     array_costs (3d array): Stack of cost rasters (not cost distance) for each source
     layer_id (integer): Index value of the layer containing the biogas source
     source_coords (tuple): Row/column of the pixel containing the biogas source (start of LCP)
     pipe_coords (tuple): Row/columns of the pipeline pixel to connect (end of LCP)
     x_size (integer): Pixel width, in units of the cost layers [default=500(m)]
     y_size (integer): Pixel height, in units of the cost layers [default=500(m)]
     
    Returns (2d array):
     A 2D array of 1s and 0s where 1s represent the least cost pathway between biogas
     source and the pipeline coordinate provided. 
    '''
    #Extract the cost surface for that farm
    arrCost = array_costs[layer_id,:,:]
    #Create the MCP graph object from the farm location
    lc_graph = graph.MCP_Geometric(arrCost, sampling=(x_size,y_size))
    #Compute the cost distance and traceback arrays
    cd_array,tb_array = lc_graph.find_costs(starts=([source_coords]))
    #Get the row/col indices of pixels in the LCP 
    lcp_indices = lc_graph.traceback(pipe_coords)
    #Convert collection of indices to a layer
    arr_LCP = np.zeros(arrCost.shape) #Create a layer of all NaNs
    for r,c in lcp_indices:                 #Loop through coordinates in traceback
        arr_LCP[r,c] = 1                     # ..set values to 1
    #Return the lcp array
    return(arr_LCP)

Find the least cost farm location occurring among the pipeline pixels. This is done by:
 1. Reducing the stack of cost distance layers to a single layer of the minimum value at each location (`arrMin`). 
 1. From this layer, pixels that don't coincide with pipeline locations are masked out (`arrMin_masked`), and 
 1. The minimum cost distance layer is extracted from the remaining pixels; this is the lowest cost pipeline pixel (`minValue`). 
 1. The row and column indices corresponding to that minimum value are determined (`rowMin` and `colMin`). 
 1. All layers in the stack of cost distance layers are searched at that row/column coordinate to determine which layer has that minimum value (`layerMin`)

In [ ]:
#Get location of the connection point
layerID, pipeCoords = getMin(array_stack=arrStack,array_mask=arrPipes,verbose=True)
#Get the location of the source for that point
srcCoords = getSourceCoordinates(array_stack=arrStack,layer_id=layerID,verbose=True)
#Create the LCP array
arrLCP = getLCP(array_costs=arrCosts,layer_id=layerID,source_coords=srcCoords,pipe_coords=pipeCoords)

In [ ]:
#View results
plt.figure(figsize = (10,10))
plt.imshow(np.ma.masked_equal(arrPipesAll,0),alpha=0.2)
plt.imshow(np.ma.masked_equal(arrLCP,0));

Next is to iterate through all the remaining layers, finding next least cost farm to connect, but this time connecting it to the least cost path deteremined above. This LCP is then added to the base LCP and the cycle continues. 

---

In [ ]:
#Drop the original layer from the stacks
arrStack_next = np.delete(arrStack,layerID,axis=0)
#Loop through the remaining layers until all layers have been processed
remaining = arrStack_next.shape[0]
while remaining > 0:
    print(arrStack_next.shape[0],end=' ')
    #Get the layer containing the next cheapest source to the nearest pipe, and the location to connect
    layerID, pipeCoords = getMin(arrStack_next,arrLCP,verbose=False)
    #Get the location of the farm associated with that layer
    farmCoords = getSourceCoordinates(arrStack_next,layerID,verbose=False)
    #Extract the LCP from the farm to the pipe coordinate
    arrLCP_next = getLCP(arrCosts,layerID,farmCoords,pipeCoords)
    #Combine the two LCP arrays
    arrLCP = np.maximum(arrLCP,arrLCP_next)
    #Remove the processed layer from the stack
    arrStack_next1=np.delete(arrStack_next,layerID,axis=0)
    arrStack_next = arrStack_next1
    #CHECK
    remaining = arrStack_next.shape[0]

In [ ]:
#Plot
#View results
plt.figure(figsize = (10,10))
plt.imshow(np.ma.masked_equal(arrPipesAll,0),alpha=0.2)
plt.imshow(np.ma.masked_equal(arrLCP,0));

In [ ]:
#Save the array
#Create the data source object
bands = 1
height,width = arrLCP.shape
drv = gdal.GetDriverByName("GTiff")
dsOut = drv.Create('../data/processed/least_cost_pipes.tif',width,height,1,gdal.GDT_Float32)
dsOut.SetGeoTransform (ds.GetGeoTransform())  #Set the pixel size, offset, and warp
dsOut.SetProjection(ds_prj)                   #Define the coordinate system


#Write to the data source object
dsOut.GetRasterBand(1).WriteArray(arrLCP)      #Write the data to the 1st band
dsOut.GetRasterBand(1).SetNoDataValue(-9999.9) #Assign NoData values
dsOut.FlushCache()